In [ ]:
import pandas as pd
import numpy as np
import pickle

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    try:
        with open(name + '.pkl', 'rb') as f:
            return pickle.load(f)    
    except FileNotFoundError as e:
        return False;

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# encounters = pd.read_csv('/content/drive/MyDrive/SIAP/data/encounters.csv')
observations = pd.read_csv('/content/drive/MyDrive/SIAP/data/observations.csv')
patients = pd.read_csv('/content/drive/MyDrive/SIAP/data/patients.csv')

obtype_borders = load_obj("/content/drive/MyDrive/SIAP/helpers/obtype_borders")

In [ ]:
observations = observations.loc[observations['TYPE'] == 'numeric']
observations['BUCKET_NAME'] = np.empty(observations.shape[0], dtype=str)

In [ ]:
def transform_row_observations(r):
    observation = r['DESCRIPTION']
    value = float(r['VALUE'])
    if observation in obtype_borders:
      borders = obtype_borders[observation]
      for i in range(1, len(borders)):
        if value <= borders[i]:
          r['BUCKET_NAME'] = 'o_' + observation + "_" + str(i-1)
          break
    r['VALUE_COUNT'] = 1
    return r

In [ ]:
observation_splits = np.array_split(observations, 10)

In [ ]:

for index, split in enumerate(observation_splits):
  observations_with_bucket_name = split.apply(transform_row_observations, axis=1)
  observations_with_bucket_name['BUCKET_NAME'].replace('', np.nan, inplace=True)
  observations_with_bucket_name = observations_with_bucket_name[observations_with_bucket_name['BUCKET_NAME'].notna()]

  observations_pivoted = pd.pivot_table(observations_with_time_weight, index=['PATIENT'], columns=['BUCKET_NAME'], values=['VALUE_COUNT'], fill_value=0)
  
  normalized_observations_pivoted.columns = normalized_observations_pivoted.columns.droplevel(0)
  break;
  normalized_observations_pivoted.to_csv('/content/drive/MyDrive/SIAP/data/prepared/observations_part_' + str(index) + '.csv', encoding='utf-8', index=True)

observations_pivoted.head(5)  

KeyError: ignored

In [ ]:
normalized_observations_pivoted.head(5)